In this part we look up Google trends data for each words

In [2]:
import datetime
from bs4 import BeautifulSoup
import requests
import pandas as pd                        
from pytrends.request import TrendReq
import numpy as np
import time

In [3]:
trend_set = pd.DataFrame(columns=['word', 'date', 'country', "value_country",'value_influence'])
list_df = []

In [6]:
data2 = pd.read_csv('C:\\Users\\franc\\Documents\\spring 2021\\social data science Proj\\ubscrape-master\\WordsToPytrend.csv')

We retrieve data for the words that we know have a set origin country, and for the words that do not have an origin country specified, but that we think are implicetely american. For both cases, we measure the influence during the time span that goes from the year previous to their submission on UD to the year later.

In [10]:
for i in (range(0 , data2.shape[0])):
    word = data2.iloc[i]['word_']
    country = data2.iloc[i]['final_countries']
    date = data2.iloc[i]['date']
    while True:
        try:
            pytrend = TrendReq()
            time1 = datetime.datetime.strptime(data2['date'][i], '%Y-%m-%d') 
            cond = (time1.month == 2) & (time1.day == 29)
            if(cond):
                timef = str(time1.replace(year = time1.year - 1, day = time1.day - 1))[0:10] + " " + str(time1.replace(year = time1.year + 1, day = time1.day - 1))[0:10]
            else:
                timef = str(time1.replace(year = time1.year - 1))[0:10] + " " + str(time1.replace(year = time1.year + 1))[0:10]            
            if(time1 < datetime.datetime(year = 2004, month = 1, day = 1)):
                timef = "2004-01-01 2004-12-31"
            if(time1 > datetime.datetime(year = 2020, month = 1, day = 1)):
                timef = "2019-01-01 2021-04-01"
            pytrend.build_payload(kw_list=[word],timeframe= timef)

            df = pytrend.interest_by_region()

            df1 = df.reset_index()
            if(df1[word].max() > 0):
                country = df1['geoName'].iloc[df1[word].argmax()]
            else:
                country = "None"

            list_df.append(df1)
            value_country = df1[word].max()
            df1= df1[df1['geoName'] != country]
            value_influence = df1[word].sum()
            trend_set.loc[i] = [word] + [date] + [country] + [value_country] + [value_influence]
            if(i % 10 == 0):
                print("We have already processed {} words".format(i))
        except requests.exceptions.Timeout:
            print("Timeout Occurred")
            time.sleep(2)
            continue
        break
            







We have already processed 0 words


In [11]:
trend_set

,word,date,country,value_country,value_influence
0,AV,2008-05-27,Norway,100,241
1,'kin 'ell,2006-08-02,None,0,0
2,'ol laid,2010-06-02,None,0,0
3,Saug,2009-10-16,Austria,100,135
4,"""ca'on""",2005-08-18,Czechia,100,658
5,FFF,2016-06-20,France,100,255
6,$100 hookup,2019-11-18,None,0,0
7,2,2013-11-15,Moldova,100,2654
8,(ry,2012-02-13,Finland,100,109
9,ca-ca,2008-02-06,Mexico,100,55


In [13]:
df_w = list_df[0].transpose().drop('geoName') 
cols =list_df[1]['geoName']
for i in range(1, len(list_df)):
    if(i % 100 == 0):
        print(i)
    df_w = pd.concat([df_w, list_df[i].transpose().drop('geoName')])
df_w.columns = list(cols)


matrix of where word i is searched by country j. Linking every word to its country of origin, we are able to create Figure 3 shown in the report.

In [14]:
df_w #(sample)

,Afghanistan,Albania,Algeria,American Samoa,Andorra,Angola,Anguilla,Antarctica,Antigua & Barbuda,Argentina,...,Vanuatu,Vatican City,Venezuela,Vietnam,Wallis & Futuna,Western Sahara,Yemen,Zambia,Zimbabwe,Åland Islands
AV,0,0,0,0,0,0,0,0,0,4,...,0,0,2,2,0,0,0,0,0,0
'kin 'ell,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
'ol laid,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Saug,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"""ca'on""",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
FFF,0,0,14,0,0,0,0,0,0,3,...,0,0,3,1,0,0,12,0,0,0
$100 hookup,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,31,0,0,0,0,0,0,34,...,0,0,27,56,0,0,0,0,0,0
(ry,0,0,0,0,0,0,0,0,0,1,...,0,0,0,3,0,0,0,0,0,0
ca-ca,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
#df_w.reset_index().drop_duplicates().to_csv("...")
#trend_set.drop_duplicates().to_csv("...")

We do the same for the non geocalized

In [15]:
trend_set = pd.DataFrame(columns=['word', 'date', 'country', "value_country",'value_influence'])
list_df = []

In [16]:
data2 = pd.read_csv('C:\\Users\\franc\\Documents\\spring 2021\\social data science Proj\\ubscrape-master\\WordsToPytrend_nongeo.csv')

In [18]:
#non geolocalized

for i in (range(0 , data2.shape[0])):
    word = data2.iloc[i]['word_id']
 #   country = data2.iloc[i]['final_countries']
    date = data2.iloc[i]['date']
    while True:
        try:
            pytrend = TrendReq()
            time1 = datetime.datetime.strptime(data2['date'][i], '%Y-%m-%d') 
            cond = (time1.month == 2) & (time1.day == 29)
            if(cond):
                timef = str(time1.replace(year = time1.year - 1, day = time1.day - 1))[0:10] + " " + str(time1.replace(year = time1.year + 1, day = time1.day - 1))[0:10]
            else:
                timef = str(time1.replace(year = time1.year - 1))[0:10] + " " + str(time1.replace(year = time1.year + 1))[0:10]            
            if(time1 < datetime.datetime(year = 2004, month = 1, day = 1)):
                timef = "2004-01-01 2004-12-31"
            if(time1 > datetime.datetime(year = 2020, month = 1, day = 1)):
                timef = "2019-01-01 2021-04-01"
            pytrend.build_payload(kw_list=[word],timeframe= timef)

            df = pytrend.interest_by_region()

            df1 = df.reset_index()
            if(df1[word].max() > 0):
                country = df1['geoName'].iloc[df1[word].argmax()]
            else:
                country = "None"

            list_df.append(df1)
            value_country = df1[word].max()
            df1= df1[df1['geoName'] != country]
            value_influence = df1[word].sum()
            trend_set.loc[i] = [word] + [date] + [country] + [value_country] + [value_influence]
            if(i % 10 == 0):
                print("We have already processed {} words".format(i))
        except requests.exceptions.Timeout:
            print("Timeout Occurred")
            time.sleep(2)
            continue
        break
            







We have already processed 0 words


In [21]:
trend_set   #of these words we'll keep only the one with country United States and value 100 - we do the filtering in R

,word,date,country,value_country,value_influence
25649,xanthrax,2010-09-02,United States,100,0
25650,xaroca,2016-03-22,Portugal,100,1
25651,Xavier Landry,2021-02-27,None,0,0
25652,XBOX Wigger,2010-08-03,None,0,0
25653,XDVandal,2005-04-19,None,0,0
25654,Xeen,2015-12-02,Iraq,100,666
0,A-Money,2012-02-23,None,0,0
1,A Bag Of Bill Murray,2009-12-05,None,0,0
2,a bit of bully,2005-02-25,United Kingdom,100,0
3,a buck 2.98,2011-07-05,None,0,0


In [19]:
df_w = list_df[0].transpose().drop('geoName') 
cols =list_df[1]['geoName']
for i in range(1, len(list_df)):
    if(i % 100 == 0):
        print(i)
    df_w = pd.concat([df_w, list_df[i].transpose().drop('geoName')])
df_w.columns = list(cols)


In [20]:
df_w

,Afghanistan,Albania,Algeria,American Samoa,Andorra,Angola,Anguilla,Antarctica,Antigua & Barbuda,Argentina,...,Vanuatu,Vatican City,Venezuela,Vietnam,Wallis & Futuna,Western Sahara,Yemen,Zambia,Zimbabwe,Åland Islands
xanthrax,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
xaroca,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Xavier Landry,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
XBOX Wigger,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
XDVandal,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Xeen,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A-Money,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A Bag Of Bill Murray,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
a bit of bully,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
a buck 2.98,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
